In [1]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import dash_pivottable
from jupyter_dash import JupyterDash
from plotly.subplots import make_subplots
import plotly.graph_objects as go

C:\Users\muser\AppData\Local\Temp\ipykernel_10168\2343935479.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\muser\AppData\Local\Temp\ipykernel_10168\2343935479.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
data = pd.read_csv("Sales.csv")
#Drop insignificant columns
data = data.drop(columns=['index','Column1'],axis=1)
data = data.dropna(axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34866 entries, 0 to 34865
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              34866 non-null  object 
 1   Year              34866 non-null  float64
 2   Month             34866 non-null  object 
 3   Customer Age      34866 non-null  float64
 4   Customer Gender   34866 non-null  object 
 5   Country           34866 non-null  object 
 6   State             34866 non-null  object 
 7   Product Category  34866 non-null  object 
 8   Sub Category      34866 non-null  object 
 9   Quantity          34866 non-null  float64
 10  Unit Cost         34866 non-null  float64
 11  Unit Price        34866 non-null  float64
 12  Cost              34866 non-null  float64
 13  Revenue           34866 non-null  float64
dtypes: float64(7), object(7)
memory usage: 4.0+ MB


In [3]:
#Set Date column
for i,date in enumerate(data["Date"]):
    data.iloc[i,0] = date[:6]+"20" + date[6:]

data["Date"] = pd.to_datetime(data["Date"])
data["Date"]

0       2016-02-19
1       2016-02-20
2       2016-02-27
3       2016-03-12
4       2016-03-12
           ...    
34861   2015-03-22
34862   2016-02-07
34863   2015-03-13
34864   2015-04-05
34865   2015-08-30
Name: Date, Length: 34866, dtype: datetime64[ns]

In [5]:
# Create a dash application
app = JupyterDash(__name__)
server = app.server

#Compute quantity and revenue by month
def choice_month(df):
    monthly_df = df.groupby(["Month","Product Category","Sub Category"])[["Quantity","Revenue"]].sum().reset_index()
    return monthly_df

#Compute quantity and revenue by day
def choice_daily(df):
    daily_df = df.groupby(["Date","Product Category","Sub Category"])[["Quantity","Revenue"]].sum().reset_index()
    return daily_df

# Application layout
app.layout = html.Div(children = [html.H1("Sales Dasboard", style= {"textAlign":"center", "font-size":40}),
                                  html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H2("Report Type: ", style={'margin-right': '2em'})
                                            ]),
                                            
                                            dcc.Dropdown(id="input-1",options=[{"label":"Monthly","value":"Monthly"},
                                                                               {"label":"Daily","value":"Daily"}])
                                                                                                    
                                        ]),

                                        html.Div([
                                            html.Div([
                                                html.H2("Product Type: ",style={"margin-right":"2em"})
                                            ]),
                                            dcc.Dropdown(id="input-2",options=[{"label":"Accessories","value":"Accessories"},
                                                                                {"label":"Clothing","value":"Clothing"},
                                                                                {"label":"Bikes","value":"Bikes"}])
                                        ]),

                                      
                                    
                                    html.Div([],id="plot-1"),
                                    html.Div([],id="plot-2"),
                                    html.Div([],id="plot-3")
                                    
                                    
                                ])
    
])

@app.callback([Output(component_id='plot-1', component_property='children'),Output(component_id='plot-2', component_property='children'),
                Output(component_id='plot-3', component_property='children')],
               [Input(component_id='input-1', component_property='value'), Input(component_id='input-2', component_property='value')])

def get_graph(chart,prod_cat):
    # Select data
    df = data[data["Product Category"]==prod_cat]

    if chart == "Monthly":
        monthly_df = choice_month(df)
        #Graph of quantity of all product sales by month
        fig_quantity = px.bar(monthly_df,x="Month",y="Quantity")
        fig_quantity.update_xaxes(categoryorder="array",categoryarray=["January", "February", "March", "April","May","June","July","August","September","October","November","December"])
        
        #Create subplot
        fig = make_subplots(rows=8,cols=1)
        #Add a quantity graph for product category's each sub category
        i = 1
        for category in monthly_df["Sub Category"].unique():
            data_categorical = monthly_df[monthly_df["Sub Category"] == category]
            fig.add_trace(go.Bar(x = data_categorical["Month"],y=data_categorical["Quantity"],name=category),row=i,col=1)
            i = i+1
        
        #Create subplot
        fig2 = make_subplots(rows=8,cols=1)
        i = 1
        #Add a revenue graph for product category's each sub category
        for category in monthly_df["Sub Category"].unique():
            data_categorical = monthly_df[monthly_df["Sub Category"] == category]
            fig2.add_trace(go.Bar(x = data_categorical["Month"],y=data_categorical["Revenue"],name=category),row=i,col=1)
            i = i+1

        #Set graph's height and title
        fig.update_layout(height=800,title_text="Graphs of Monthly Quantity")
        #Set graph's x axis order
        fig.update_xaxes(categoryorder="array",categoryarray=["January", "February", "March", "April","May","June","July","August","September","October","November","December"])
        fig2.update_layout(height=800,title_text="Graphs of Monthly Revenue")
        
        return(dcc.Graph(figure=fig_quantity),dcc.Graph(figure=fig),dcc.Graph(figure=fig2))

    else:
        daily_df = choice_daily(df)
        fig_quantity = px.line(daily_df,x="Date",y="Quantity")

        #Create subplot
        fig = make_subplots(rows=8,cols=1)
        #Add a quantity graph for product category's each sub category
        i = 1
        for category in daily_df["Sub Category"].unique():
            data_categorical = daily_df[daily_df["Sub Category"] == category]
            fig.add_trace(go.Line(x = data_categorical["Date"],y=data_categorical["Quantity"],name=category),row=i,col=1)
            i = i+1

        #Create subplot
        fig2 = make_subplots(rows=8,cols=1)
        i = 1
        #Add a quantity graph for product category's each sub category
        for category in daily_df["Sub Category"].unique():
            data_categorical = daily_df[daily_df["Sub Category"] == category]
            fig2.add_trace(go.Bar(x = data_categorical["Date"],y=data_categorical["Revenue"],name=category),row=i,col=1)
            i = i+1

        #Set graph's height and title
        fig.update_layout(height=800,title_text="Graphs of Daily Quantity")
        fig2.update_layout(height=800,title_text="Graphs of Daily Revenue")
        
        return(dcc.Graph(figure=fig_quantity),dcc.Graph(figure=fig),dcc.Graph(figure=fig2))

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


c:\Users\muser\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


